In [1]:
import pandas as pd
from os import walk
import numpy as np
import torch
import random
from sys import stdout
from refit_data import *
from scipy import stats, integrate
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

In [5]:
path="/home/ibcn079/data/CLEAN_REFIT_081116/"
houses = [1,2]
#homes = OrderedDict()
#home=read_all_homes(path, houses)

data = load_csv(path, columns_rename, 1,  tz="Europe/London")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [11]:
df = s = pd.Series(home)
df

1                               Aggregate  Fridge  ...
2                               Aggregate  Fridge-F...
3                               Aggregate  Toaster ...
dtype: object

In [4]:
def convert_to_tensor(aggregate, individual, window_size):
    """Method for converting aggregate and individual power usage pandas.Dataframes to two torch 2D tensors of the same size so aggregate data is input and
        individual data is desired output.
        Args:
                aggregate: pandas.Dataframe containing concatenated and downsampled aggregate channels data
                individual: pandas.Dataframe containing signal from individual appliance channels
                NOTE: the lengths of aggregate and individual data must be the same.
        return aggregate, individual : Two torch Tensors with data grouped in windows of specified size
    """

    #Converting to numpy array with usage only (dropping Time column completely).
    aggregate = np.array(aggregate)
    individual = np.array(individual)

    #Creating padding so the last row of torch.Tensor can fit to window size of specifiedAppliance
    zeros_padding = window_size - len(aggregate)%window_size

    #Appending zeros padding to the end of numpy arrays of both individual and aggregate
    aggregate = np.append(aggregate, np.zeros(zeros_padding))
    individual = np.append(individual, np.zeros(zeros_padding))

    #Conversion to 1D torch.Tensor from numpy
    aggregate = torch.from_numpy(aggregate)
    individual = torch.from_numpy(individual)

    #Reshaping from 1D to 2d toch.Tensor
    aggregate = aggregate.view(-1, window_size)
    individual = individual.view(-1, window_size)

    return aggregate, individual


In [5]:
total = home['1'].Aggregate
fridge = home['1'].Fridge
window = 599

In [6]:
aggregate, individual=convert_to_tensor(aggregate=total, individual=fridge, window_size=window)

In [7]:
aggregate


   523    526    540  ...     212    210    210
   212    212    212  ...     900    900   1257
  1409   1129   1129  ...     255    253    251
        ...            ⋱           ...         
   181    187    181  ...     184    184    182
   184    181    187  ...     181    183    188
   188    180    182  ...       0      0      0
[torch.DoubleTensor of size 11620x599]

In [8]:
def generate_clean_data2(data_dir, appliance, window_size, threshold, proportion=[1,1],test=False, test_on='All'):
    activation_proportion = proportion[0]
    non_activation_proportion = proportion[1]
    aggregate_channels = []
    individual_channels = []
    aggregate_channels_test = []
    individual_channels_test = []
    activations = []
    non_activations = []
    houses = []
    if appliance == 'Fridge':
        houses = ['h1'] #houses from which fridge data is read
    elif appliance =='Washing Machine':
        houses = ['h1','h2'] #houses from which fridge data is read
    for house in houses: 
        
    